# Tutorial (0): Interfacing Qulacs and OpenFermion

There are several useful functions to accelerate algorithm developments.


Quket extends the usabilities of `openfermion`'s `FermionOperator` and `QubitOperator` such that they can be directly applied to `Qulacs`' `QuantumState`.  
It also adds some easy-manipulations for `QuantumState` so that it can be added or subtracted.  

In [1]:
# Import necessary modules
from quket import *

mpi4py is not imported. no MPI.


#### `QuntumState`, `QubitOperator`, `FermionOperator` used in `Quket` are different from those with original libaries. 
These classes are overridden and defined in `quket.lib` module to enable these functionalities.  

In [2]:
import quket, openfermion, qulacs
print('QuantumState')
print(quket.QuantumState)
print(qulacs.QuantumState)

print('\nQubitOperator')
print(quket.QubitOperator)
print(openfermion.QubitOperator)

print('\nFermionOperator')
print(quket.FermionOperator)
print(openfermion.FermionOperator)

QuantumState
<class 'quket.lib.qulacs.QuantumState'>
<class 'qulacs.QuantumState'>

QubitOperator
<class 'quket.lib.openfermion.QubitOperator'>
<class 'openfermion.ops.operators.qubit_operator.QubitOperator'>

FermionOperator
<class 'quket.lib.openfermion.FermionOperator'>
<class 'openfermion.ops.operators.fermion_operator.FermionOperator'>


# QuantumState

`quket.QuantumState` prepares a quantum state as a state vector, in exactly the same way as `qulacs.QuantumState`. 

The following example generates a 4-qubit state in `psi`. <br>
`psi` initially comprises only `|0000>`, and can be set to `|0011>` by `set_computational_basis`.

In [3]:
psi = QuantumState(4)
print_state(psi)
psi.set_computational_basis(0b0011)
print_state(psi)

  Basis          Coef
| 0000 > : +1.0000 +0.0000i

  Basis          Coef
| 0011 > : +1.0000 +0.0000i



However, `quket.QuantumState` allows to prepare this state with the following line.

In [4]:
psi = QuantumState(4, '|0011>')
print_state(psi)

  Basis          Coef
| 0011 > : +1.0000 +0.0000i



The second argument accepts either of the followings:
- a determinant as `str` or `int` : e.g. `'|0011>'`, `3`
- a linear combination of determinants as `str` : e.g. `'-0.1 * |0011> + (0.2 + 0.5j) |1100>'`
- Haar random state by `'random'`

if `normalize == False`, no normalization is performed.

In [5]:
psi1 = QuantumState(4, 3)
print_state(psi1, '3')

psi2 = QuantumState(4, '-0.1 * |0011> + (0.2 + 0.5j) |1100>')
print_state(psi2, '-0.1 * |0011> + (0.2 + 0.5j) |1100>')

psi3 = QuantumState(4, 'random')
print_state(psi3, 'random')

3
  Basis          Coef
| 0011 > : +1.0000 +0.0000i

-0.1 * |0011> + (0.2 + 0.5j) |1100>
  Basis          Coef
| 0011 > : -0.1826 +0.0000i
| 1100 > : +0.3651 +0.9129i

random
  Basis          Coef
| 0000 > : -0.0586 +0.1279i
| 0001 > : +0.0193 +0.3091i
| 0010 > : +0.3305 +0.1649i
| 0011 > : -0.3023 -0.3309i
| 0100 > : +0.1724 +0.2381i
| 0101 > : -0.0224 +0.1333i
| 0110 > : -0.1372 +0.1663i
| 0111 > : +0.1802 +0.1610i
| 1000 > : +0.1143 -0.2005i
| 1001 > : +0.0041 -0.1116i
| 1010 > : -0.0802 -0.2628i
| 1011 > : +0.1688 -0.0833i
| 1101 > : +0.0902 +0.1601i
| 1110 > : +0.1961 -0.0169i
| 1111 > : +0.2140 +0.1868i



## Multiplication by a scalar
`Quket`'s `QuantumState` can be multiplied by a scalar value.

In [6]:
print_state((-1 + 0.8j) * psi)

  Basis          Coef
| 0011 > : -1.0000 +0.8000i



## Addition and subtraction between different `QuantumState`s

In [7]:
print_state(psi + psi2, 'psi + psi2')
print_state(psi - psi2, 'psi - psi2')

psi + psi2
  Basis          Coef
| 0011 > : +0.8174 +0.0000i
| 1100 > : +0.3651 +0.9129i

psi - psi2
  Basis          Coef
| 0011 > : +1.1826 +0.0000i
| 1100 > : -0.3651 -0.9129i



## Inner product
Inner product can be performed by `@`. This uses `qulacs`'s `inner_product`.

```
  complex = QuantumState @ QuantumState
```

In [8]:
print(psi2 @ psi3)
print(inner_product(psi2, psi3))

(0.14001045598169598+0.06999431435649374j)
(0.14001045598169598+0.06999431435649374j)


## Applying QubitOperator

In some cases, one may want to apply an arbitrary non-unitary operator, for example $\hat H$, to a quantum state.  
Such an operation is not trivial with quantum computers but the state-vector implementation of Quket allows us to do it.  

Application of an operator (or scalar value) to a qunatum state can be performed by `@`

```
  QuantumState = QubitOperator @ QuantumState
```

In [9]:
Op = QubitOperator('X0 Y1 Z2') + QubitOperator('X1 Y2 Z3', 1j)
psi = QuantumState(4, '|0011>')

Hpsi = Op @ psi
print_state(Hpsi)

  Basis          Coef
| 0000 > : +0.0000 -1.0000i
| 0101 > : -1.0000 +0.0000i



If the ordering of QubitOperator and QuantumState is opposite,
```
  QuantumState = QuantumState @ QubitOperator
```
still gives `QuantumState` but `QubitOperator` is Hermitian conjugated to present a bra state.

In [10]:
psiH = psi @ Op
print_state(psiH)

  Basis          Coef
| 0000 > : +0.0000 -1.0000i
| 0101 > : +1.0000 +0.0000i



Some examples:<br>
(note that `Op` is not Hermitian here)

In [11]:
print(psi @ Op @ psi3)
print(psiH @ psi3)
print(psi3 @ Hpsi)
print(psi3 @ Op @ psi)

(-0.15025680202288694+0.07466325771041324j)
(-0.15025680202288694+0.07466325771041324j)
(-0.10552866567584013+0.19193008711920465j)
(-0.10552866567584013+0.19193008711920465j)


## In order to use these features, the following functions should be replaced by those of Quket.

```python
from quket import(
    QubitOperator,
    FermionOperator,
    jordan_wigner,
    reverse_jordan_wigner,
    bravyi_kitaev,
    get_fermion_operator,
    commutator,
    s_squared_operator,
    number_operator,
    normal_ordered,
    hermitian_conjugated
    )
```

For more details, including `quket.FermionOperator`, please refer to Tutorial 7.